<a href="https://colab.research.google.com/github/minnji88/DeepLearning2020/blob/master/8_1_LSTM%EC%9C%BC%EB%A1%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 8.1.4 글자 수준의 LSTM 텍스트 생성 모델 구현

언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 책에서는 19세기 후반의 독일 철학자 니체의 글(영문)을 이용합니다. 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주체를 따르는 모델일 것입니다.

### 데이터 전처리

- 먼저 말뭉치를 내려받아 소문자로 바꿉니다.

In [ ]:
# 코드 8-2 원본 텍스트 파일을 내려받아 파싱하기
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('length  of corpus : ', len(text))

length  of corpus :  600893


그 다음 maxlen 길이를 가진 시퀀스를 중복하여 추출합니다. 추출된 시퀀스를 원-핫 인코딩으로 변환하고 (sequences, maxlen, unique_characters)인 3D 넘파이 배열 x로 합칩니다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 y를 준비합니다. 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자입니다.

In [ ]:
# 코드 8-3 글자 시퀀스 벡터화하기

# 60개의 글자로 된 시퀀스를 추출합니다
maxlen = 60
step = 3    # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링합니다.

sentences = [] # 추출한 시퀀스를 담을 리스트
next_chars = [] # 타깃을 담을 리스트

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])

print('Number of sequences : ', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자: ', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorize...')
# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
x = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sequences :  200278
고유한 글자:  57
Vectorize...


## 네트워크 구성

이 네트워크는 하나의 LSTM 층과 그 뒤에 Dense 분류기가 뒤따릅니다. 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듭니다. 순환 신경망이 시퀀스 데이터를 생성하는 유일한 방법은 아닙니다. 최근에는 1D 컨브넷도 이런 작업에 아주 잘 들어맞는다는 것이 밝혀졌습니다. (6장)

In [ ]:
# 코드 8-4 다음 글자를 예측하기 위한 단일 LSTM 모델
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

타깃이 원-핫 인코딩되어 있기 때문에 모델을 훈련하기 위해 categorical_crossentropy 손실을 사용합니다.

categorical_crossentropy : 네트워크(신경망)가 출력한 확률 분포와 진짜 레이블의 분포 사이의 거리, 두 확률 분포 사이의 거리를 측정한다.

두 분포 사이의 거리를 최소화하면서 진짜 레이블에 가능한 가까운 출력을 내도록 모델을 훈련한다.

In [ ]:
# 코드 8-5 모델 컴파일 설정하기
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 언어 모델 훈련과 샘플링
훈련된 모델과 시드(seed)로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
특정 온도로 이 확률 분포의 가중치를 조정합니다.
가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
새로운 글자를 생성된 텍스트의 끝에 추가합니다.
다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다.

In [ ]:
# 코드 8-6 모델의 예측이 주어졌을 떄 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

마지막으로 반복적으로 훈련하고 텍스트를 생성하는 반복문입니다. 에포크마다 학습이 끝난 후 여러 가지 온도를 사용하여 텍스트를 사용합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트를 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여줍니다.

In [ ]:
# 코드 8-7 텍스트 생성 루프
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
# for epoch in range(1, 60):
for epoch in range(1, 10):
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1) # 데이터에서 한 번만 반복해서 모델을 학습합니다

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 261s 165ms/step - loss: 2.2450
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the possible the partion of the pain and instinction of the supposition of the restince of the pain of the same the partion of such a man is the same the constious and the paste itself, the passion of the partion of the pain of the suppere of the same the believe of the pains and instince of the pain of the self--the spirit and the sould and the passion of the passion of the partion of the pain 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for to the ratrey and lately the the conscience, and the restince of the devilation
and the be knows would of the enessions of himself of the all the have distruscity and virtue the resurding and is so late the rase of passion of the insures of the self-the conscience of the stranges of the ch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


g of the suffering
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the acts of the soul of the philosophy in the life of superfical entangable as a the all the sufferers of the all reference of himself in the sense of the present the princing and literable and internations of the same the sense of the continent and abstines the delight to sense of the part of the suffering in the politics of the frage to really higher and from the democratic men when he would n
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through for one periloved for eur
shart of feithority--thus proved, and gods as a is
naturar.

1on henceagation even a new comture
and velloce ensibleing of thope froith attenes deceived the functions for oner serse. yse, topatherunally shal and they asonnstorry of the result of an immence, the obliges of it is thencalmotness of human to case in his take th sluance topar in the a
fualor, an uncall be mensle
------ 온도: 1.2
the slowly asc

결과를 보면 알 수 있지만, 온도가 낮을 때는 문장의 완성도가 비교적 높습니다. 실제 존재하는 단어와 비교적 안정적인 구조, 반복적이고 예상되는 텍스트를 만듭니다. 하지만 높은 온도의 경우 문장의 구조가 무너지며, 꽤 그럴싸하게 보이는 완전히 새로운 단어를 만들어냅니다. (ex.mestoped) **텍스트 생성에 가장 좋은 온도는 중간인 0.5정도가 적당합니다.**

더 많은 데이터에서 크고 깊은 모델을 훈련하면 이보다 더 좋은 실제와 같은 텍스트 샘플을 만들 수 있습니다. 하지만 우연이 아닌 의미 있는 텍스트 생성은 무리가 있습니다.